In [78]:
import pandas as pd
import numpy as np
train_df=pd.read_csv('../input/nlp-getting-started/train.csv')
test_df=pd.read_csv('../input/nlp-getting-started/test.csv')
sub_df=pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

In [2]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
train_df=train_df.drop(columns=['id','keyword','location'],axis=1)
train_df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import transformers
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = TFBertModel.from_pretrained("bert-base-uncased")

2022-07-15 08:56:34.661004: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-07-15 08:56:34.661109: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

2022-07-15 08:56:55.343545: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-15 08:56:55.346215: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-07-15 08:56:55.346245: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-15 08:56:55.346272: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (0089dc043f39): /proc/driver/nvidia/version does not exist
2022-07-15 08:56:55.347598: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

In [6]:
print('max lenth of word is',max([len(x.split()) for x in train_df['text']]))

max lenth of word is 31


In [54]:
max_length=35

In [18]:
tokenizer

PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [19]:
bert

In [20]:
c='he is a bad man'
tokenizer(c)

{'input_ids': [101, 2002, 2003, 1037, 2919, 2158, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [55]:
x_train=tokenizer(

    text=train_df['text'].tolist(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_attention_mask=True,
    verbose=True
    
)

In [56]:
x_train

{'input_ids': <tf.Tensor: shape=(7613, 35), dtype=int32, numpy=
array([[  101,  2256, 15616, ...,     0,     0,     0],
       [  101,  3224,  2543, ...,     0,     0,     0],
       [  101,  2035,  3901, ...,     0,     0,     0],
       ...,
       [  101, 23290,  1012, ...,  3501,   102,     0],
       [  101,  2610, 11538, ...,     0,     0,     0],
       [  101,  1996,  6745, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(7613, 35), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(7613, 35), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32

In [57]:
import tensorflow as tf
from tensorflow.keras import Model,layers,optimizers,callbacks,losses,metrics,utils

In [58]:
x_train.attention_mask

<tf.Tensor: shape=(7613, 35), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>

In [60]:
input_ids=layers.Input(shape=(max_length,),dtype=tf.int32,name='input_ids')
attention_mask=layers.Input(shape=(max_length,),dtype=tf.int32,name='attention_mask')
embedding=bert(input_ids,attention_mask)[1]

In [61]:
out=layers.Dropout(0.1)(embedding)
out=layers.Dense(399,activation='relu')(out)

out=layers.Dropout(0.15)(out)
out=layers.Dense(67,activation='relu')(out)
out=layers.Dense(17,activation='relu')(out)
y=layers.Dense(1,activation='sigmoid')(out)

nn=Model(inputs=[input_ids,attention_mask],outputs=y)
nn.layers[2].trainable=True
optimizer=optimizers.Adam(learning_rate = 1e-5)

In [62]:
y_train=train_df['target']
y_train

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [76]:
nn.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
nn.fit(

    x={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']},
    y=y_train,
    validation_split=0.20,
    epochs=1,
    batch_size=45

)

136/136 [==============================] - 1182s 9s/step - loss: 0.1841 - accuracy: 0.9409 - val_loss: 0.4758 - val_accuracy: 0.8345


# Working with test data

In [79]:
test_df=test_df.drop(columns=['id','keyword','location'],axis=1)
test_df.head()

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan


In [80]:
x_test=tokenizer(

    text=test_df.text.tolist(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_attention_mask=True,
    verbose=True


)

In [81]:
x_test.attention_mask.shape

TensorShape([3263, 35])

In [82]:
predicted=nn.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

In [83]:
y_predicted=np.where(predicted>0.5,1,0)
y_predicted

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [84]:
y_predicted=y_predicted.reshape(-1,)
sub=pd.DataFrame({'id':sub_df['id'],'target':y_predicted})
sub.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [85]:
sub['target'].value_counts()

0    2003
1    1260
Name: target, dtype: int64

In [86]:
sub.to_csv('bert_nlp_4.csv',index=None)